In [1]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

## XGBoost Cloud Prediction Invocation Template

Invoke SageMaker Prediction Service

In [2]:
import boto3
import re
import sagemaker
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
# Acquire realtime endpoint
endpoint_name = 'xgboost-biketrain-vl'
predictor = sagemaker.predictor.Predictor(endpoint_name = endpoint_name)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
#from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer, CSVDeserializer
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

predictor.content_type = "text/csv"
predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()
predictor.accept = CONTENT_TYPE_JSON


In [5]:
df_all = pd.read_csv('bike_test.csv')
df_all.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4


In [6]:
df_all.columns[1:]

Index(['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed', 'year', 'month', 'day', 'dayofweek', 'hour'],
      dtype='object')

In [7]:
df_all.drop(columns = ['datetime'], inplace=True)
df_all.head(4)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour
0,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0
1,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1
2,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2
3,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3


In [8]:
# Need to pass an array to the prediction
# can pass a numpy array or a list of values [[19,1], [20,1]]
#arr_test = df_all.to_numpy()

arr_test = df_all[df_all.columns[1:]].values

In [56]:
df_all[df_all.columns[1:]].values

array([[ 1.,  0.,  1., ...,  3.,  0., 14.],
       [ 1.,  0.,  1., ...,  3.,  1., 11.],
       [ 1.,  0.,  1., ...,  3.,  2., 10.],
       ...,
       [ 1.,  0.,  1., ...,  0., 21., 33.],
       [ 1.,  0.,  1., ...,  0., 22., 37.],
       [ 1.,  0.,  1., ...,  0., 23., 37.]])

In [9]:
arr_test

array([[ 0.,  1.,  1., ..., 20.,  3.,  0.],
       [ 0.,  1.,  1., ..., 20.,  3.,  1.],
       [ 0.,  1.,  1., ..., 20.,  3.,  2.],
       ...,
       [ 0.,  1.,  1., ..., 31.,  0., 21.],
       [ 0.,  1.,  1., ..., 31.,  0., 22.],
       [ 0.,  1.,  1., ..., 31.,  0., 23.]])

In [10]:
type(arr_test)

numpy.ndarray

In [11]:
arr_test.shape

(6493, 12)

In [12]:
arr_test[:5]

array([[0.00000e+00, 1.00000e+00, 1.00000e+00, 1.06600e+01, 1.13650e+01,
        5.60000e+01, 2.60027e+01, 2.01100e+03, 1.00000e+00, 2.00000e+01,
        3.00000e+00, 0.00000e+00],
       [0.00000e+00, 1.00000e+00, 1.00000e+00, 1.06600e+01, 1.36350e+01,
        5.60000e+01, 0.00000e+00, 2.01100e+03, 1.00000e+00, 2.00000e+01,
        3.00000e+00, 1.00000e+00],
       [0.00000e+00, 1.00000e+00, 1.00000e+00, 1.06600e+01, 1.36350e+01,
        5.60000e+01, 0.00000e+00, 2.01100e+03, 1.00000e+00, 2.00000e+01,
        3.00000e+00, 2.00000e+00],
       [0.00000e+00, 1.00000e+00, 1.00000e+00, 1.06600e+01, 1.28800e+01,
        5.60000e+01, 1.10014e+01, 2.01100e+03, 1.00000e+00, 2.00000e+01,
        3.00000e+00, 3.00000e+00],
       [0.00000e+00, 1.00000e+00, 1.00000e+00, 1.06600e+01, 1.28800e+01,
        5.60000e+01, 1.10014e+01, 2.01100e+03, 1.00000e+00, 2.00000e+01,
        3.00000e+00, 4.00000e+00]])

In [13]:
result = predictor.predict(arr_test[:2])

In [14]:
result

{'predictions': [{'score': 2.770601511001587}, {'score': 2.5148653984069824}]}

In [15]:
len(result['predictions'])

2

In [17]:
for i in range(len(result['predictions'])):
 print(result['predictions'][i]['score'])

2.770601511001587
2.5148653984069824


## Split the input data into chunks

There are thousands of rows in this data set for which need inference.
When communicating over internet, it is a good idea to split the data into chunks to prevent payload and timeout error

In [19]:
# For large number of predictions we can split the input data and
# query the prediction service
# array_split is convenient to specify how many splits are needed
predictions = []
for arr in np.array_split(arr_test,10):
    result = predictor.predict(arr)
    for i in range(len(result['predictions'])):
        predictions.append(result['predictions'][i]['score'])
#     result = result.decode("utf-8")
#     result = result.split(',')
#     print(arr.shape)
    #print(result)
     #predictions += [float(r) for r in result]

In [22]:
len(predictions)

6493

In [23]:
arr_test.shape

(6493, 12)

In [37]:
type(arr_test)

numpy.ndarray

In [38]:
type(predictions)

list

In [24]:
predictions[1:10]

[2.5148653984069824,
 2.46087384223938,
 2.824162721633911,
 3.434159517288208,
 3.683140993118286,
 3.5565006732940674,
 3.5565006732940674,
 3.652454137802124,
 3.557997465133667]

In [39]:
res = np.expm1(predictions)

In [40]:
type(res)

numpy.ndarray

In [49]:
# reuploading the file since we dropped th edatetime column
df_all = pd.read_csv('bike_test.csv')

In [50]:
df_all['count'] = res

In [51]:
df_all.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour,count
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0,14.968236
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1,11.364944
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2,10.715044
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3,15.846834
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4,30.005342


In [52]:
type(df_all['count'])

pandas.core.series.Series

In [53]:
df_all['count']= df_all['count'].astype('int')

In [54]:
df_all['count'] = df_all['count'].round()
df_all.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour,count
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0,14
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1,11
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2,10
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3,15
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4,30


In [55]:
df_all[['datetime', 'count']].to_csv('predicted_count_cloud.csv', index = False)

In [ ]:
# Delete Endpoint to prevent unnecessary charges
predictor.delete_endpoint()

### Clean Up

To avoid incurring unnecessary charges, use the AWS Management Console to delete the endpoints and resources that you created while running the exercises.

    1.Open the Amazon SageMaker console at https://console.aws.amazon.com/sagemaker/ and delete the following resources:

        The endpoint. 
        Deleting the endpoint also deletes the ML compute instance or instances that support it.

        Under Inference, choose Endpoints.

        Choose the endpoint that you created in the example, choose Actions, and then choose Delete.

        The endpoint configuration.

        Under Inference, choose Endpoint configurations.

        Choose the endpoint configuration that you created in the example, choose Actions, and then choose Delete.

        The model.

        Under Inference, choose Models.

        Choose the model that you created in the example, choose Actions, and then choose Delete.

        The notebook instance. 
        Before deleting the notebook instance, stop it.

        Under Notebook, choose Notebook instances.

        Choose the notebook instance that you created in the example, choose Actions, and then choose Stop. The notebook instance takes several minutes to stop. When the Status changes to Stopped, move on to the next step.

        Choose Actions, and then choose Delete.

    2.Open the Amazon S3 console at https://console.aws.amazon.com/s3/, and then delete the bucket that you created for storing model artifacts and the training dataset.

    3.Open the Amazon CloudWatch console at https://console.aws.amazon.com/cloudwatch/, and then delete all of the log groups that have names starting with /aws/sagemaker/.

Note:
Training jobs and logs cannot be deleted and are retained indefinitely.
If you plan to explore other exercises in this guide, you might want to keep some of these resources, such as your notebook instance, S3 bucket, and IAM role.